In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import bisect
from scipy.stats import norm

file_path = './SPX_hedging.csv'
data = pd.read_csv(file_path)

In [8]:
data.describe()

,ID,Days until next hedge,S,Dividend,C_BS,D_BS,C_mkt,D_Blm,R,TTM,Moneyness,D_Optimal,Target,K
count,85.0,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.00000,85.000000,85.000000,85.000000,84.000000,85.0
mean,1.0,1.411765,5718.486941,1.345927,300.508027,0.591168,315.221176,0.609341,4.80564,168.870588,18.458824,0.060418,-0.532742,5700.0
std,0.0,0.806139,201.911555,0.047218,93.940648,0.120690,95.263399,0.120187,0.10632,35.091305,201.966260,5.258367,5.295591,0.0
min,1.0,1.000000,5186.330000,1.276500,106.223538,0.291429,111.850000,0.312000,4.59072,108.000000,-514.000000,-39.819644,-40.533411,5700.0
25%,1.0,1.000000,5597.120000,1.311000,237.004250,0.526772,257.450000,0.549000,4.76095,140.000000,-103.000000,0.424070,-0.145064,5700.0
50%,1.0,1.000000,5728.800000,1.336400,295.415563,0.591379,311.800000,0.610000,4.77997,169.000000,29.000000,0.580349,-0.006258,5700.0
75%,1.0,1.000000,5853.980000,1.372100,368.601090,0.669328,383.750000,0.686000,4.88116,198.000000,154.000000,0.765608,0.156792,5700.0
max,1.0,3.000000,6049.880000,1.481600,466.011822,0.800804,487.300000,0.818000,5.01499,228.000000,350.000000,8.794966,8.216118,5700.0


In [16]:
# Black-Scholes call option pricing formula
def black_scholes_call_price(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# Function to calculate implied volatility
def implied_volatility(C_mkt, S, K, T, r):
    def objective(sigma):
        return black_scholes_call_price(S, K, T, r, sigma) - C_mkt
    try:
        return bisect(objective, 1e-6, 5)  # Searching for sigma in a reasonable range
    except ValueError:
        return np.nan

# Function to calculate Delta using the Black-Scholes model
def black_scholes_delta(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return norm.cdf(d1)

# Crank-Nicholson method for solving the Black-Scholes PDE
def crank_nicholson(S, K, r, T, sigma, N=100, M=100):
    dt = T / N  # Time step size
    dx = sigma * np.sqrt(3 * dt)  # Price step size
    pu = 0.5 * dt * ((sigma / dx)**2 + (r - 0.5 * sigma**2) / dx)
    pm = 1 - dt * (sigma / dx)**2 - r * dt
    pd = 0.5 * dt * ((sigma / dx)**2 - (r - 0.5 * sigma**2) / dx)

    # Initialize the price grid
    S_grid = S * np.exp(dx * (np.arange(-M, M+1)))
    print(S_grid)
    V = np.maximum(S_grid - K, 0)  # Option value at maturity

    # Backward iteration to solve the PDE
    for _ in range(N):
        V[1:-1] = pu * V[:-2] + pm * V[1:-1] + pd * V[2:]
        V[0] = 0  # Boundary condition at S = 0
        V[-1] = S_grid[-1] - K  # Boundary condition at S -> infinity
    return V[M]



# In[ ]:






In [17]:
# Initialize strategy variables
position = 0  # Current asset position
cash = 0      # Cash balance
portfolio_values = []  # Store portfolio values over time
predicted_prices = []  # Store predicted option prices

# Iterate over each row of the data to implement the hedging strategy
for i in range(len(data)):
    S = data['S'][i]  # Current stock pricex
    K = data['K'][i]  # Calculate strike price based on Moneyness
    T = data['TTM'][i] / 252  # Convert time to expiration to years
    r = data['R'][i] / 100  # Convert interest rate to decimal
    C_mkt = data['C_mkt'][i]  # Market option price

    # Calculate implied volatility
    sigma = implied_volatility(C_mkt, S, K, T, r)

    # Skip this row if implied volatility could not be calculated
    if np.isnan(sigma):
        continue

    # Calculate Delta using Crank-Nicholson if needed
    V_up = crank_nicholson(S * 1.001, K, r, T, sigma, 1000)
    V_down = crank_nicholson(S * 0.999, K, r, T, sigma, 1000)
    delta = (V_up - V_down) / (S * 0.02)

    # Calculate predicted option price using Crank-Nicholson
    predicted_price = crank_nicholson(S, K, r, T, sigma)
    predicted_prices.append(predicted_price)

    # Determine the target position based on Delta
    target_position = -delta
    position_change = target_position - position

    # Update cash and asset position
    cash -= position_change * S
    position = target_position

    # Record portfolio value
    portfolio_values.append(position * S + cash)

[ 2666.62210191  2684.44680174  2702.39064853  2720.4544387
  2738.63897401  2756.94506155  2775.37351383  2793.92514878
  2812.60078981  2831.4012658   2850.32741121  2869.38006605
  2888.56007597  2907.86829225  2927.30557188  2946.87277755
  2966.57077775  2986.40044675  3006.36266467  3026.45831753
  3046.68829725  3067.05350173  3087.55483485  3108.19320655
  3128.96953285  3149.88473589  3170.93974397  3192.1354916
  3213.47291954  3234.95297484  3256.57661085  3278.34478734
  3300.25847047  3322.31863286  3344.52625362  3366.88231843
  3389.38781954  3412.04375584  3434.8511329   3457.81096299
  3480.92426518  3504.19206533  3527.61539617  3551.19529731
  3574.93281533  3598.8290038   3622.88492334  3647.10164164
  3671.48023355  3696.02178109  3720.72737351  3745.59810736
  3770.6350865   3795.83942217  3821.21223306  3846.7546453
  3872.46779258  3898.35281616  3924.41086493  3950.64309544
  3977.05067199  4003.63476667  4030.39655938  4057.33723793
  4084.45799805  4111.76004

In [15]:
# Convert portfolio values to a DataFrame for further analysis
portfolio_values = pd.DataFrame(portfolio_values, columns=['Portfolio Value'])

# Print predicted prices
predicted_prices_df = pd.DataFrame(predicted_prices, columns=['Predicted Price'])
print(predicted_prices_df.head())
print("---------------")
print(data.head()['C_mkt'])

# Calculate risk metrics
portfolio_values['Returns'] = portfolio_values['Portfolio Value'].pct_change().dropna()
volatility = portfolio_values['Returns'].std()  # Calculate return volatility
cumulative_returns = (1 + portfolio_values['Returns']).cumprod()
drawdown = cumulative_returns.cummax() - cumulative_returns  # Calculate drawdown
max_drawdown = drawdown.max()
# Output risk metrics
volatility, max_drawdown

   Predicted Price
0        49.671126
1        30.594648
2        27.676281
3        39.703143
4        31.635428
---------------
0    147.20
1    125.90
2    111.85
3    153.75
4    145.55
Name: C_mkt, dtype: float64


/Users/jacksparrow/NYU_schoolwork/2024/quant_methods/Crank-Nicholson_delta_hedging/venv/lib/python3.13/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


(np.float64(nan), np.float64(nan))